In [1]:
%load_ext lab_black

# Distributed training example using PyTorch

This notebook trains, evaluates and deploys a classifier for handwritten digits (Using the MNIST dataset). The training is distributed across multiple GPUs using DDP.


Author: Nick Lawrence ntl@us.ibm.com
License: [Apache 2.0](https://www.apache.org/licenses/LICENSE-2.0)

## Base container Image
The components of this pipeline use a a container image that has been built for this example. The Dockerfile is included in the [container_image](./container_image) directory. 

The container image does *not* include a notebook server, meaning you cannot use the base image for a notebook server. A new container image was created so that PyTorch 2.0 and Python 3.10 could be used in the example. These packages were not available when the notebook server images were built. The image also includes pytorch-lightning, which makes implementing a model for distributed training simpler.

The image also has installed the pytorch_distributed_kf_tools, which simplifies the creation and deployment of the PyTorch Job. The source code for this package is in [../../distributed_kf_tools](../../distributed_kf_tools).

The base image does not include the python code for the PyTorch model. This is cloned from github into a pvc, you can find the source [in the ./src directory](./src)

## Workspace
In addition to using parameters to pass information between components, a shared workspace is mounted to many of the components.

The workspace is used to:
* Store python implementation of the model and training code (after downloading from github)
* Store training and test data for the MNIST dataset (downloaded from pytroch)
* Store checkpoints created during the training process
* Store tensorboard logs that are created during training

## Imports and Constants

In [2]:
import kfp
from kfp.components import InputPath, OutputPath
import kfp.dsl as dsl
from kfp.dsl import PipelineConf, data_passing_methods
import numpy as np
import os
from typing import NamedTuple, Dict

BASE_IMAGE = "quay.io/ntlawrence/pytorch:1.0.1"
COMPONENT_CATALOG_FOLDER = f"{os.getenv('HOME')}/components"

## Node Selectors

An important observation of GPUs on Power 9 nodes is that each node has only one type of GPU. These constants are used to ensure that pipeline tasks or inference servers run on the correct type of node.

In [3]:
GPU_NODE_LABEL = "nvidia.com/gpu.product"
INFERENCE_GPU_PRODUCT = "Tesla-V100-SXM2-32GB"
TRAINING_GPU_PRODUCT = "Tesla-V100-SXM2-32GB"

# Define the component that prepares the workspace

This component prepares the shared workspace by:
* downloading the training and test data
* cloning the github repo that contains the python code for model and training
* creating the directory for the tensorboard logs

In [4]:
def prepare_workspace(
    github_repo: str = "https://github.com/ntl-ibm/kubeflow-ppc64le-examples.git",
    branch: str = "pytorch_example_improvements",
    workspace_mount_point: str = "/workspace",
) -> NamedTuple(
    "CreatedPaths", [("data", str), ("code", str), ("tensorboard_logs", str)]
):
    """
    Prepares the workspace with a directory structure

    params:
        github_repo - repo to clone
        branch - branch or tag from the repo to clone

    returns:
        Tuple of paths that can be referenced by later components (The mount point is not included in the path)
          data - where the data has been downloaded
          code - where the repo has been cloned into
          tensorboard_logs - empty directory for storing tensorboard logs
    """

    import subprocess
    import os
    from collections import namedtuple

    from torchvision.datasets import MNIST

    # Download data
    _ = MNIST(f"{workspace_mount_point}/data", download=True, train=True)

    # Clone git repo
    subprocess.run(
        f"git clone {github_repo}  {workspace_mount_point}/code -b {branch}",
        shell=True,
        check=True,
    )

    os.mkdir(f"{workspace_mount_point}/tensorboard_logs")

    out_tuple = namedtuple("CreatedPaths", ["data", "code", "tensorboard_logs"])

    # These are the paths withing the PVC that caller cares about
    return out_tuple("data", "code", "tensorboard_logs")


prepare_workspace_comp = kfp.components.create_component_from_func(
    prepare_workspace, base_image=BASE_IMAGE
)

# Define a component to create a tensorboard

Our training code logs data to a tensorboard, which graphs training metrics in real time as the training progresses.

The "Visualizations" tab of the component provides a link to the TensorBoard service.

In [5]:
CONFIGURE_TENSORBOARD_COMPONENT = f"{os.getenv('HOME')}/kubeflow-ppc64le-examples/configure_tensorboard_component/configure_tensorboard_component.yaml"

configure_tensorboard_comp = kfp.components.load_component_from_file(
    CONFIGURE_TENSORBOARD_COMPONENT
)

# Define a component to train the model

This example uses a pytorch job with DDP to perform distributed training.

The component creats a pytorch job that manages the workers performing distributed training. A diagram of this setup is shown here. The tasks in the kubeflow pipeline are on the right, the PyTorchJob custom resource is in the middle, and the worker jobs are shown on the left.


![Diagram of pytorch job](./pytorchjob.jpg)

The component uses a run_pytorch_job python method to work with the pytorch job, the python package is built into the container image, with the source code being available [here](../distributed_kf_tools).
The run_pytorch_job method performs these basic tasks:
* Construct pytorch job template
* Create pytorch job resource
* Wait until the resource indicates that training is complete
* Report errors if training fails
* Terminate training if the pipeline is terminated.
* Stream training logs to the train_model task's logs.

Our training reoutine includes checkpointing. Checkpoints are written to the workspace at each epoch, if an error occurs, training is resumed from the previous checkpoint. The training component will fail if more than 5 retries occur.

The pytorch job is cleaned up at the end of training.

Metrics are logged to the tensorboard, and can be viewed in real time.

The workspace pvc name is passed as a parameter, so that the PyTorchJob can mount that PVC on each worker. The path to the best model's checkpoint is returned as an output parameter from the train_model task.

In [6]:
def train_model(
    workspace_pvc_name: str,
    worker_image: str,
    tensorboard_path: str,
    data_path: str,
    working_dir: str,
    num_workers: int = 3,
    batch_size: int = 672,
    max_epochs: int = 50,
    node_selector: Dict[str, str] = None,
) -> NamedTuple("ModelCkptPath", [("best", str)]):

    import distributed_kf_tools.deploy as deploy
    from distributed_kf_tools.template import OwningWorkFlow, PvcMount
    import subprocess
    from collections import namedtuple

    workspace_mount_point = "/workspace"

    success = False
    for retries in range(5):
        try:
            ## Start the PyTorch job for distributed training
            job_name = "{{workflow.name}}" + (f"-{retries:03d}" if retries else "")
            deploy.run_pytorch_job(
                # owning_workflow setups it up so that when the pipeline is deleted,
                # the training job is cleaned up
                owning_workflow=OwningWorkFlow(
                    name="{{workflow.name}}", uid="{{workflow.uid}}"
                ),
                # These place holders for namespace and job name are
                # filled in by Kubeflow when the pipeline runs.
                namespace="{{workflow.namespace}}",
                pytorch_job_name=job_name,
                # Shared volumes used by the training script
                pvcs=[
                    PvcMount(
                        pvc_name=(workspace_pvc_name),
                        mount_path=workspace_mount_point,
                    )
                ],
                working_dir=f"{workspace_mount_point}/{working_dir}",
                # The command to run in each worker
                # This starts with "torch.distributed.run" for DDP
                # The output model path needs to be stored on the pvc,
                # The output path for this POD points the tmp directory of THIS pod,
                # Storing it in the temp directory of the PyTorch worker pod doesn't work,
                # because this pod doesn't have access to that.
                command=[
                    "python",
                    "-m",
                    "torch.distributed.run",
                    "train.py",
                    "--root_dir=./work",
                    f"--data_dir={workspace_mount_point}/{data_path}",
                    f"--model_ckpt={workspace_mount_point}/models/best.ckpt",
                    f"--batch_size={batch_size}",
                    "--checkpoint",
                    "--early_stopping",
                    f"--max_epochs={max_epochs}",
                    "--pytorchjob",
                    f"--tensorboard={workspace_mount_point}/{tensorboard_path}",
                ],
                # Number of workers (pods)
                num_workers=num_workers,
                # Number of GPUs per worker (OK to leave this at 1)
                gpus_per_worker=1,
                # The base image used for the worker pods
                worker_image=worker_image,
                node_selector=node_selector,
            )

            success = True
            break
        except RuntimeError as e:
            print(f"THE JOB FAILED BECAUSE OF ERROR {e}")

    if not success:
        raise RuntimeError("Training has failed")

    out_tuple = namedtuple("ModelCkptPath", ["best"])
    return out_tuple("models/best.ckpt")


train_model_comp = kfp.components.create_component_from_func(
    train_model,
    base_image=BASE_IMAGE,
    packages_to_install=[
        "git+https://github.com/ntl-ibm/kubeflow-ppc64le-examples@3.0.0#subdirectory=distributed_training/distributed_kf_tools"
    ],
)

# Define a component to test the model

This runs the python test script.

The metrics from the testing are returned as an output parameter (mlpipeline_metrics). This will allow the metrics to appear on the experiment page. This figure shows the acc (accuracy) and F1 metrics as columns in the experiment runs window.

![metrics](./experiment_metrics.jpg)


In [7]:
def test_model(
    data_path: str,
    working_dir: str,
    model_path: str,
    workspace_mount_point: str = "/workspace",
    batch_size: int = 224,
) -> NamedTuple("EvaluationOutput", [("mlpipeline_metrics", "Metrics")]):
    import json
    from collections import namedtuple
    import subprocess

    subprocess.run(
        [
            "python",
            "test.py",
            "--root_dir=/tmp",
            f"--data_dir={workspace_mount_point}/{data_path}",
            f"--model_ckpt={workspace_mount_point}/{model_path}",
            "--batch_size=672",
            "--metrics_json=/tmp/metrics.json",
        ],
        cwd=f"{workspace_mount_point}/{working_dir}",
        check=True,
    )

    with open("/tmp/metrics.json", "r") as f:
        metrics = json.load(f)

    metrics = {
        "metrics": [
            {"name": "acc", "numberValue": metrics["test/acc"], "format": "RAW"},
            {"name": "F1", "numberValue": metrics["test/F1"], "format": "RAW"},
        ]
    }

    out_tuple = namedtuple("EvaluationOutput", ["mlpipeline_metrics"])
    return out_tuple(json.dumps(metrics))


test_model_comp = kfp.components.create_component_from_func(
    test_model, base_image=BASE_IMAGE
)

# Define a component to convert the model to ONNX

ONNX is a standard formant for Neural Networks. In this pipeline, we convert to ONNX because we have an inference server (Triton) that can handle ONNX models. It's easier to convert to ONNX and use Triton than to build a PyTorch Inference Service.

When a pytorch model is converted to ONNX, the conversion involves both static analysis of the python code, and also tracing of a reference data set through the model.  The mechanism to do this is part of pytorch, and can be accomplished with a simple function call.

As with the training and test process, we've contained that logic in a python script.

In [8]:
def convert_to_onnx(
    onnx_model: OutputPath(str),
    data_path: str,
    working_dir: str,
    model_path: str,
    workspace_mount_point: str = "/workspace",
):
    import shutil
    import os
    from pathlib import Path
    import subprocess

    subprocess.run(
        [
            "python",
            "convert.py",
            "--root_dir=/tmp",
            f"--data_dir={workspace_mount_point}/{data_path}",
            f"--model_ckpt={workspace_mount_point}/{model_path}",
            "--onnx=/tmp/model.onnx",
        ],
        cwd=f"{workspace_mount_point}/{working_dir}",
        check=True,
    )

    directory = str(Path(onnx_model).parent.absolute())
    os.makedirs(directory, exist_ok=True)
    shutil.copy("/tmp/model.onnx", onnx_model)


convert_to_onnx_comp = kfp.components.create_component_from_func(
    convert_to_onnx, base_image=BASE_IMAGE
)

# Define a component to upload the model to MinIO.

Storing models in object storage is very common, MinIO is an Object Store that comes with KubeFlow.

In [9]:
UPLOAD_MODEL_COMPONENT = (
    f"{COMPONENT_CATALOG_FOLDER}/model-building/upload-model/component.yaml"
)

upload_model_comp = kfp.components.load_component_from_file(UPLOAD_MODEL_COMPONENT)

# Define a component to depoly the Inference Service

We use Triton in this example. This example is NOT setup to use a GPU for inferencing.

In [10]:
DEPLOY_INFERENCE_SERVICE_COMPONENT = f"{os.getenv('HOME')}/kubeflow-ppc64le-examples/deploy_triton_inference_service_component/deploy_triton_inference_service_component.yaml"

deploy_inference_service_comp = kfp.components.load_component_from_file(
    DEPLOY_INFERENCE_SERVICE_COMPONENT
)

# Define the pipeline

In [11]:
PIPELINE_NAME = "Mnist-HW"

In [12]:
@dsl.pipeline(
    name=PIPELINE_NAME,
    description="An example pipeline that trains using distributed pytorch",
)
def mnist_pipeline(
    github_repo: str = "https://github.com/ntl-ibm/kubeflow-ppc64le-examples.git",
    branch: str = "3.0.0",
    context: str = "distributed_training/pytorch/mnist/src",
):

    workspace_volume_volop = dsl.VolumeOp(
        name="Create workspace for training",
        resource_name="shared-workspace-pvc",
        modes=dsl.VOLUME_MODE_RWM,
        size="4Gi",
        set_owner_reference=True,
    )

    prepare_workspace_task = prepare_workspace_comp(
        github_repo, branch, workspace_mount_point="/workspace"
    )
    prepare_workspace_task.add_pvolumes({"/workspace": workspace_volume_volop.volume})

    configure_tensorboard_task = configure_tensorboard_comp(
        pipeline_name=PIPELINE_NAME,
        pvc_name=workspace_volume_volop.volume.persistent_volume_claim.claim_name,
        pvc_path=prepare_workspace_task.outputs["tensorboard_logs"],
    )

    train_model_task = train_model_comp(
        workspace_pvc_name=workspace_volume_volop.volume.persistent_volume_claim.claim_name,
        worker_image=BASE_IMAGE,
        tensorboard_path=prepare_workspace_task.outputs["tensorboard_logs"],
        data_path=prepare_workspace_task.outputs["data"],
        working_dir=f"{prepare_workspace_task.outputs['code']}/{context}",
        node_selector={GPU_NODE_LABEL: TRAINING_GPU_PRODUCT},
    )
    train_model_task.after(configure_tensorboard_task)

    test_model_task = test_model_comp(
        data_path=prepare_workspace_task.outputs["data"],
        working_dir=f"{prepare_workspace_task.outputs['code']}/{context}",
        model_path=train_model_task.outputs["best"],
        workspace_mount_point="/workspace",
    )
    test_model_task.add_pvolumes({"/workspace": workspace_volume_volop.volume})

    convert_to_onnx_task = convert_to_onnx_comp(
        data_path=prepare_workspace_task.outputs["data"],
        working_dir=f"{prepare_workspace_task.outputs['code']}/{context}",
        model_path=train_model_task.outputs["best"],
        workspace_mount_point="/workspace",
    )
    convert_to_onnx_task.add_pvolumes({"/workspace": workspace_volume_volop.volume})

    model_name = "mnist"
    minio_url = "minio-service.kubeflow:9000"

    upload_model_task = upload_model_comp(
        convert_to_onnx_task.outputs["onnx_model"],
        minio_url=minio_url,
        export_bucket="{{workflow.namespace}}-models",
        model_format="onnx",
        model_name=model_name,
        model_version=1,
    )

    deploy_model_task = deploy_inference_service_comp(
        name=model_name,
        rm_existing=True,
        storage_uri="s3://{{workflow.namespace}}-models/onnx/",
        minio_url=minio_url,
        concurrency_target=4,  # soft limit, may be exceeded for short periods of time
        predictor_min_replicas=0,  # min_replicas supports scale to 0
        predictor_max_replicas=4,  # We don't want to scale till we consume all the available GPUs
        predictor_protocol="v2",
    )
    deploy_model_task.after(upload_model_task)

# Configure Pipeline

The first transformer disables caching for our pipeline. Kubeflow caches tasks based on input and output parameters. Because we are using shared storage, the input parameters are the same as previous runs, however the data on the shared storage is most likely different.


In [13]:
pipeline_conf = kfp.dsl.PipelineConf()

# Disable Caching
def disable_cache_transformer(op: dsl.ContainerOp):
    if isinstance(op, dsl.ContainerOp):
        op.execution_options.caching_strategy.max_cache_staleness = "P0D"
    else:
        op.add_pod_annotation(
            name="pipelines.kubeflow.org/max_cache_staleness", value="P0D"
        )
    return op


pipeline_conf.add_op_transformer(disable_cache_transformer)

# Compile, upload and run pipeline

This creates a run of the pipeline within an experiment.

Once the pipeline is uploaded, you can run it from the Kubeflow UI with parameters. Here we run the pipeline programatically, so othat we can wait for it to finish before running other cells to demo inference.


In [14]:
def delete_pipeline(pipeline_name: str):
    """Delete's a pipeline with the specified name"""

    client = kfp.Client()
    existing_pipelines = client.list_pipelines(page_size=999).pipelines
    matches = (
        [ep.id for ep in existing_pipelines if ep.name == pipeline_name]
        if existing_pipelines
        else []
    )
    for id in matches:
        client.delete_pipeline(id)


def get_experiment_id(experiment_name: str) -> str:
    """Returns the id for the experiment, creating the experiment if needed"""
    client = kfp.Client()
    existing_experiments = client.list_experiments(page_size=999).experiments
    matches = (
        [ex.id for ex in existing_experiments if ex.name == experiment_name]
        if existing_experiments
        else []
    )

    if matches:
        return matches[0]

    exp = client.create_experiment(experiment_name)
    return exp.id

In [15]:
PIPELINE_NAME = "train-mnist-pytorch-model"

client = kfp.Client()
kfp.compiler.Compiler().compile(
    pipeline_func=mnist_pipeline,
    package_path=f"{PIPELINE_NAME}.yaml",
    pipeline_conf=pipeline_conf,
)

delete_pipeline(PIPELINE_NAME)
uploaded_pipeline = client.upload_pipeline(f"{PIPELINE_NAME}.yaml", PIPELINE_NAME)
run = client.run_pipeline(
    experiment_id=get_experiment_id("mnist-pytorch-exp"),
    job_name="mnist-pytorch",
    pipeline_id=uploaded_pipeline.id,
    params={},
)

# Wait for pipeline completion

The complention report includes both the status of the run, and also metrics.

In [16]:
TWENTY_MIN = 20 * 60
result = client.wait_for_run_completion(run.id, timeout=TWENTY_MIN)
{
    "status": result.run.status,
    "error": result.run.error,
    "time": str(result.run.finished_at - result.run.created_at),
    "metrics": result.run.metrics,
}

{'status': 'Succeeded',
 'error': None,
 'time': '0:11:38',
 'metrics': [{'format': 'RAW',
   'name': 'acc',
   'node_id': 'mnist-hw-jtl64-3979504309',
   'number_value': 0.9797000288963318},
  {'format': 'RAW',
   'name': 'F1',
   'node_id': 'mnist-hw-jtl64-3979504309',
   'number_value': 0.9795413017272949}]}

# Make a prediction

This section uses the inference service to make a prediction.

Our inference service is setup to use a REST protocol, this is not as fast as a GRPC protocol, but it is easier to work with since the input and output data is JSON.

In [17]:
from PIL import Image
import numpy as np

## Load image

Here we display the image, and convert to an array.

In [18]:
with Image.open("3.jpg") as i:
    i.show()
    image_array = np.asarray(i)

## Shape

The shape of the loaded image is 28x28, however our model is built with a channels dimension, even though we only have a single channel.

We also need to add a batch dimension to get the shape to be (1,1,28,28).

In [19]:
image_array.shape

(28, 28)

In [20]:
# channels dimension is required, even though the shape is always (1,28,28)
tensor = np.expand_dims(image_array, 0)

In [21]:
# batch dimension required
tensor = np.expand_dims(tensor, 0)
tensor.shape

(1, 1, 28, 28)

## Get service details

We can verify the inference service has been created and is ready with an OC command

In [22]:
!oc get inferenceservice mnist

NAME    URL                                                     READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION             AGE
mnist   http://mnist-ntl-us-ibm-com.apps.openshift.sopenshift   True           100                              mnist-predictor-default-00001   43s


### url

With a little code, we can get the URL for the inference API (v2 format in this example)

In [23]:
import subprocess
import yaml
import requests

p = subprocess.run(
    "oc get inferenceservice mnist -o yaml", shell=True, stdout=subprocess.PIPE
)

desc = yaml.safe_load(p.stdout)
url = desc["status"]["address"]["url"]
predictor = desc["status"]["components"]["predictor"]["address"]["url"]

## Model details

Trition offers an extension to tell us about the model, specifically the input names and data types.

In [24]:
requests.get(f"{predictor}/v2/models/mnist/versions/1").json()

{'name': 'mnist',
 'versions': ['1'],
 'platform': 'onnxruntime_onnx',
 'inputs': [{'name': 'input.1', 'datatype': 'FP32', 'shape': [1, 1, 28, 28]}],
 'outputs': [{'name': '18', 'datatype': 'FP32', 'shape': [1, 10]}]}

## Make request

Once we know the name, shape and datatype of the input, we can make an inference request.

In [25]:
import requests
import json

payload = {
    "inputs": [
        {
            "name": "input.1",
            "shape": list(tensor.shape),
            "datatype": "FP32",
            "data": tensor.flatten().tolist(),
        }
    ]
}

res = requests.post(url, headers={}, data=json.dumps(payload))

## The highest score is the winner

In [26]:
np.argmax(res.json()["outputs"][0]["data"])

3